## 함수 II
- 중첩함수
- 제너레이터
- 재귀함수

### 중첩함수
- 함수를 포함한 함수
- 캡슐화 목적
    - 변수 범위 제한 가능
    - 책임, 관리 명확해짐
- 변수에 할당하여 사용 가능

In [5]:
def outer(a, b): #외부함수
    def inner(c, d): #내부함수
        return c + d
    return inner(a, b)

In [6]:
outer(2, 2)

4

In [7]:
inner(2, 2) # error => 선언한 적이 없다.

NameError: name 'inner' is not defined

In [8]:
def knights(saying):
    def inner(quote):
        return f'we are the knights who say: {quote}'
    return inner(saying)

In [9]:
knights('Ni!')

'we are the knights who say: Ni!'

클로저
- 자신을 둘러싼 scope (name space)의 상태값을 기억하는 함수
- 메모리 효율적 사용 (함수 호출 시 꺼내 쓸 수 있음)
- 조건
    - 중첩함수여야 함
    - 외부함수의 상태값 참조해야 함
    - 외부함수가 내부함수를 반환해야 함

In [20]:
def multiply(x): #클로저 (closure)
    def inner(y):
        return x * y
    return inner #함수 실행되기 전

In [21]:
multiply

<function __main__.multiply(x)>

In [22]:
m5 = multiply(5) # x=5
m6 = multiply(6) # x=6

In [23]:
m5(6), m6(9)

(30, 54)

In [24]:
del(multiply)

In [25]:
multiply

NameError: name 'multiply' is not defined

In [27]:
m5(10) # multiply가 삭제되어도 실행됨

50

### 데코레이터
- 메인 함수에 또다른 함수를 데코레이터로 선언하여 사용할 수 있음
- `@`가 달린 함수의 기능이 모두 메인 함수로 들어감
- 목적:
    - 재사용, 가독성, 직관적

In [30]:
def document_it(func):
    def new_func(*args, **kargs):
        print('arguments: ', args)
        print('key arguments: ', kargs)
        return func(*args, **kargs)
    return new_func

@document_it
def add(a, b):
    return a + b

@document_it
def subtract(a, b):
    return a - b

In [31]:
add(1, 3)

arguments:  (1, 3)
key arguments:  {}


4

In [32]:
# add에 활용할 중첩함수 만들기
# 결과값의 제곱 값을 반환하는 클로저 함수 만들기

def square(func):
    def new_func(*args):
        result = func(*args)
        return result * result
    return new_func

@square
def add(a, b):
    return a + b

In [33]:
add(3, 4) # 7*7

49

#### scope: global, local, nonlocal
- 내부함수는 외부함수의 인자를 "참조"(읽기)만 할 수 있음
    - 계산 등 불가
    - nonlocal 예약어를 활용하여 해결

In [37]:
# 전역 (global)
def square(func):
    # 지역 (local)
    def new_func(*args):
        # 지역 내의 지역(?)
        result = func(*args)
        return result * result
    return new_func

In [38]:
z = 3
def outer(x):
    y = 4
    def inner():
        x = 1000
        return x
    return x

outer(10)
# inner을 타지 않았으므로 그대로 10

10

In [39]:
z = 3
def outer(x): # x값 참조
    y = 4
    def inner():
        x = 1000
        return x
    return inner()

outer(10)
# inner을 탔으므로 무엇을 넣든 1000

1000

In [42]:
z = 3
def outer(x):
    y = 4
    def inner():
        nonlocal x
        x += 1
        return x
    return inner()

outer(10)

11

In [44]:
# 좋지 않은 함수의 예
def my_func(nums:list): # 가변인자를 넣었는데, 리턴 없이도 리턴 얻음
    nums.append(sum(nums))

a = [1, 2, 3]
my_func(a)

In [47]:
a

[1, 2, 3, 6]

In [49]:
my_func(a)
my_func(a)
my_func(a)
a

[1, 2, 3, 6, 12, 24, 48, 96, 192]

### 실습
1. 함수
    - 차 속도, 제한 속도를 비교해서 True / False
2. 데코레이터 함수
    - 만약 제한 속도를 초과했다면 얼마나 초과했는지 프린트하는 함수
    - example) 100, 80 --> "20 km/h 초과"

In [50]:
def calc_speed(func):
    def inner(speed, limit):
        if func(speed, limit):
            return f'초과 속도: {speed - limit} km/h'
        else:
            return f'정상 속도'
    return inner

@calc_speed
def is_speeding(speed, limit):
    return speed > limit

is_speeding(100, 80)

'초과 속도: 20 km/h'

### 제너레이터
- return (X) / yield (O)
- 순회의 리턴값을 하나씩 변환
- 시퀀스를 생성하는 객체
- 메모리 효율성 증대
    - 한번 사용되고 메모리에서 사라짐
- 함수를 for문으로 돌려줘야 함

In [60]:
def example():
    for i in range(5):
        yield i

for i in example():
    print(i)

0
1
2
3
4


In [76]:
names = 'Kevin Michael Juliette Laura'.split()

def printing(name_list:list):
    for name in name_list:
        yield name

name_list = printing(names)

In [77]:
for i in printing(names):
    print(i)

for i in printing(names):
    print(i)

Kevin
Michael
Juliette
Laura
Kevin
Michael
Juliette
Laura


In [78]:
for i in name_list:
    print(i)

for i in name_list:
    print(i) #두 번째는 출력 X

Kevin
Michael
Juliette
Laura


#### 실습
- range 함수 구현하기

In [81]:
# 내 답안
def my_range(start, end, step):
    temp_num = start
    while temp_num < end:
        yield temp_num
        temp_num += step

ranger = my_range(1, 10, 2)
for i in ranger:
    print(i)

1
3
5
7
9


In [83]:
# 모법 답안
def my_range(start, end, step=1):
    while start < end:
        yield start
        start += step

ranger = my_range(1, 10, 2)
for i in ranger:
    print(i)

1
3
5
7
9


### 재귀함수
- 자기 자신을 호출하는 함수
- 모든 요소의 차원을 단일화시킬 때 사용
- 재귀가 너무 깊어지면 run time error 발생 주의

In [86]:
def flatten(sent):
    for word in sent:
        if isinstance(word, list): #리스트가 맞다면
            for sub_word in flatten(word):
                yield sub_word
        else:
            yield word

a = [1, 2, [2, 3, 4], [[[1, 2]]]]

In [87]:
for i in flatten(a):
    print(i)

1
2
2
3
4
1
2


In [90]:
def flatten(sent):
    for word in sent:
        if isinstance(word, list):
            yield from flatten(word) # 파이썬 3.3부터 가능
        else:
            yield word

In [91]:
isinstance('word', int)

False

## 예외 처리 | exception handling
- 프로그램 동작 중 예외가 발생했을 때 대처하기 위함
- 사용자에게 예외를 알리고, 원하는 조치를 설정함
- 프로그램이 정상적으로 종료가 될 수 있음
- 가장 흔한 에러 종류
    - `ZeroDivisionError`: 0으로 나누었을 때
    - `IndexError`: 입력한 index가 범위 밖일 때
    - `ValueError`: 형 변환이 불가능할 때
    - `NameError`: 정의되지 않은 변수를 불러올 때
- 예외가 발생할 수도 있는 코드 블럭을 try문에 넣음
    - 전체를 넣으면 오류 시 전체가 끝나버림

In [92]:
try:
    # 예외가 발생할 수도 있는 코드 블럭
    5/0
except ZeroDivisionError:
    # 에외 시 실행할 것
    print('0으로 나눌 수 없음')

0으로 나눌 수 없음


In [95]:
# 전체를 try문에 넣기
try:
    for i in range(10):
        print(10 / i)
except ZeroDivisionError:
    print('0으로 나눌 수 없음')

0으로 나눌 수 없음


In [96]:
# 예외가 발생할 수도 있는 코드 블럭만 넣기
for i in range(10):
    try:
        print(10 / i)
    except ZeroDivisionError:
        print('0으로 나눌 수 없음')

0으로 나눌 수 없음
10.0
5.0
3.3333333333333335
2.5
2.0
1.6666666666666667
1.4285714285714286
1.25
1.1111111111111112


In [99]:
word = 'hello'
while True:
    index = input('인덱스를 입력하세요>> ')
    if index == 'q':
        break
    try:
        index = int(index) # ValueError, IndexError
        print(word[index])
    # 단순히 에러의 모습을 보고 싶을 때 사용하는 방법
    except ValueError as e1:
        print(e1)
    except IndexError as e2:
        print(e2)

인덱스를 입력하세요>>  0


h


인덱스를 입력하세요>>  10


string index out of range


인덱스를 입력하세요>>  x


invalid literal for int() with base 10: 'x'


인덱스를 입력하세요>>  q


### 예외 일으키기
- 프로그램을 강제 종료시키기 위해 사용함
- raise, assert
    - `raise <예외타입>(<메세지>)`
    - `assert <참인 조건>, <False일 경우 메세지>`
        - AssertionError로 발생

In [100]:
while True:
    num = input('숫자>> ')
    if not num.isdigit():
        raise ValueError('숫자가 아닙니다!')
    else:
        print(int(num) + 5)

숫자>>  2


7


숫자>>  ㅇ


ValueError: 숫자가 아닙니다!

In [101]:
def get_binary(num):
    assert isinstance(num, int), '정수 아님'
    return bin(num)

get_binary('10')

AssertionError: 정수 아님

#### 사용자 정의 예외 타입
- class 선언, Exception 클래스를 상속 받는다.

In [102]:
class MyException(Exception):
    pass

In [106]:
cities = 'dublin newyork seoul TOKYO'.split()

In [107]:
# 예외에 대해 메세지를 따로 정해주고 싶을 때 방법
for city in cities:
    if city.isupper():
        raise MyException('대문자 안됨')
    else:
        print(city)

dublin
newyork
seoul


MyException: 대문자 안됨

In [108]:
# 메세지를 아예 고정하고 싶을 때

class UppercaseException(Exception):
    def __init__(self):
        super().__init__('대문자 안된다구')

for city in cities:
    if city.isupper():
        # raise MyException('대문자 안됨')
        raise UppercaseException
    else:
        print(city)

dublin
newyork
seoul


UppercaseException: 대문자 안된다구